# The first of three gcode .ipynb files:  

    gcode_DRO ...
    gcode_Part2.ipynb
    gcode_Part3.ipynb

## There are 3 sets of user inputs in this file.


##### Remember ....   <<-   is used to create a global variable

In [1]:

Turn_Data <- function() {
    StartDia <<- readline("What is Stock Diameter?")
    EndDia <<- readline("What is the Finish Diameter?")
    Cut <<- readline("What is the depth of the cut? Aluminum .015-.020 ")
    Leng <<- readline("What is the length of the cut along the negative -Z axis?")
    Spin <<- readline("What is the Spindle Speed ... max 2800?")
    Feed <<- readline("What is the Feedrate ... 4 is slow, 10 fast?")
    
    # Convert input strings to numeric values
    StartDia <<- as.numeric(unlist(strsplit(StartDia, ", ")))
    EndDia <<- as.numeric(unlist(strsplit(EndDia, ", ")))
    EndDia <- format(EndDia, nsmall = 3)
    Cut <<- as.numeric(unlist(strsplit(Cut, ", ")))
    Leng <<- as.numeric(unlist(strsplit(Leng, ", ")))
    Spin <<- as.numeric(unlist(strsplit(Spin, ", ")))
    Feed <<- as.numeric(unlist(strsplit(Feed, ", ")))
}


# Input:

In [2]:
Turn_Data()

What is Stock Diameter? .4925
What is the Finish Diameter? .2545
What is the depth of the cut? Aluminum .015-.020  .015
What is the length of the cut along the negative -Z axis? 1.0
What is the Spindle Speed ... max 2800? 2800
What is the Feedrate ... 4 is slow, 10 fast? 5


In [3]:
is.numeric(Cut)
Cut

[1] TRUE

[1] 0.015

#### Know: In a lathe, the cut size (Cut) reduces the diameter 2 x Ct, the radius is reduced 1/2 x Ct

## Lathe gcodes


    G07 - lathe radius mode
    G18 - the ZX plane mode
    G20 - Set units to inches
    G49 - Set tool offset compensation to zero (used w/Fusion360 )
    G90 - Distance Mode: ABSOLUTE
    
    Opt
    G53 - Safe position (Non Modal)
    G54 - Touchoff position for X and Z
    G94 - Feed per minute.  Sherline cannot change spindle speed, t/f, cannot use G95
    

In [4]:
X <- "X "
Z <- "Z "

In [5]:
X

[1] "X "

In [6]:
# Preserve Cut value
CutSize <- Cut
CutSize

[1] 0.015

In [7]:
Remove <- StartDia - EndDia     # the TOTAL amount of matterial to be removed 
Radius <- StartDia / 2        
Diameter <- StartDia

Remove
Radius
Diameter


CutDia <- CutSize * 2       # Reduction in diameter for each cut
CutRad <- CutSize / 2       # Reduction in radius for each cut

CutDia
CutRad


CutCount <- Remove / Cut  # To determine the number of cuts to reduce the stock toward finished diameter

CutCount            # S/B 16.6
MajorCuts <- trunc(CutCount)
MajorCuts
FinishCut <- CutCount %% 1
FinishCut
FinishCut * Cut


[1] 0.238

[1] 0.24625

[1] 0.4925

[1] 0.03

[1] 0.0075

[1] 15.86667

[1] 15

[1] 0.8666667

[1] 0.013

#####  Know: In this lathe app, the cut size (Cut) is reducing the diameter by the Cut size, and the radius is reduced xx x Cut

    Ex.  Start diameter = .500
         Cut = .015
         Remaining diameter = .500 - .015 = .485



In [8]:
# need gcode to change for each toolpath along the Z axis - reposition the X by the Cut
# there will be count for number of cuts turning the part down to size 
# Need a var to keep the diminishing Stock diameter  (diameter - CutD )
# need a Radius var, and need to keep track of the diminishing radius ( Radius - CutR)
# Need an if stmt to ensure the material isn't being cut below EndDia
# Need to calc the measured value of the left over (.333333 ) from the CutCount 
# Need a For stmt to cycle each cut


# Down and Dirty - just gitter done!

# Move the cutter tip to just touch the outside edge of the part, then reset the X G54 touchoff to zero (use LinuxCNC t/o X button)
# Next, reset the Distance Mode to G91 (INCREMENTAL)
# The plan:  make -X moves by the desired cut size (.015 - .02 for Aluminum)

# Assign a variable.  This assisgns the depth of the cut.     #100 = -.015


if (Leng >= 0) 
    Leng <- Leng *-1;



ChangingDiameter <- StartDia 

     #Initialize an empty dataframe to store the values
        result_df <- data.frame(X = numeric(0), Z = numeric(0))


for (i in 1:CutCount) {
    

    
    # Create a new row for each iteration with X and Z values and add it to the dataframe
      new_row <- data.frame(X = ChangingDiameter, Z = Leng)
      result_df <- rbind(result_df, new_row)
   
      print(paste("X", format(ChangingDiameter, nsmall = 3), "Z", format(Leng, nsmall = 2)))
    
      ChangingDiameter <- ChangingDiameter - Cut   

    }

DecisionCut <- ChangingDiameter + Cut



[1] "X 0.4925 Z -1.00"
[1] "X 0.4775 Z -1.00"
[1] "X 0.4625 Z -1.00"
[1] "X 0.4475 Z -1.00"
[1] "X 0.4325 Z -1.00"
[1] "X 0.4175 Z -1.00"
[1] "X 0.4025 Z -1.00"
[1] "X 0.3875 Z -1.00"
[1] "X 0.3725 Z -1.00"
[1] "X 0.3575 Z -1.00"
[1] "X 0.3425 Z -1.00"
[1] "X 0.3275 Z -1.00"
[1] "X 0.3125 Z -1.00"
[1] "X 0.2975 Z -1.00"
[1] "X 0.2825 Z -1.00"


In [9]:
DecisionCut



[1] 0.2825

In [10]:
result_df

X,Z
<dbl>,<dbl>
0.4925,-1
0.4775,-1
0.4625,-1
0.4475,-1
0.4325,-1
0.4175,-1
0.4025,-1
0.3875,-1
0.3725,-1


In [11]:
n <- nrow(result_df)
n
result_df$X[n]


[1] 15

[1] 0.2825

In [12]:
.015*8

Radius

RadRemainingDRO <- Radius - (.015 * 8)
RadRemainingDRO    # Agrees with the last X from For Loop


RadDROzero <- Radius / 2       #  The perm/final Rad value based on the toolpath


RadRemainingDRO - RadDROzero



[1] 0.12

[1] 0.24625

[1] 0.12625

[1] 0.003125

### This is the remaining stock to be cut.  Agrees with RadRemainingDRO


RadDRO <- Radius /2  - RadRemainingDRO
RadDRO


In [13]:
RemainingCutSize <- result_df$X[n]
RemainingCutSize <- format(RemainingCutSize, nsmall = 3 )
RemainingCutSize


[1] "0.2825"

In [14]:
Cut
str(Cut)

[1] 0.015

 num 0.015


## Part 1 - Decision: To make more rough cut of Cut size?


In [15]:
valid_input <- FALSE

while (!valid_input) {
  # Prompt the user for input
    
RemainingCutSize <- format(RemainingCutSize, nsmall = 3)
    
    user_input <<- readline(paste("The stock diameter has been reduced to: " , RemainingCutSize, ": The roughing cut size is:", Cut, ". The final diameter will be:", format(EndDia, nsmall = 3), "\n", "Enter Y to make one last rough cut. Or enter N to go directly to final cut options."))
    
    # Convert input to uppercase for case-insensitive comparison
  user_input <- toupper(user_input)
  
  # Check if the input is 'Y' or 'N'
  if (user_input == 'Y') {
    print("You entered Yes (Y)")
      
   Cut <- as.numeric(Cut)
    
    RemainingCutSize <- as.numeric(RemainingCutSize)
      
   RemainingCutSize <- RemainingCutSize - Cut     # Update diameter after last rough cut         
      
      
    # Create a new row for each iteration with X and Z values and add it to the dataframe
    new_row <- data.frame(X = RemainingCutSize, Z = Leng)            
    print(paste("The new diameter is ", new_row))
    Output_gcode_df <<- rbind(result_df, new_row)
              
    valid_input <- TRUE  # Set valid_input to TRUE to exit the loop
      
  } else if (user_input == 'N') {
    print("You entered No (N)")
    Output_gcode_df <<- result_df
    valid_input <- TRUE  # Set valid_input to TRUE to exit the loop
  } else {
    print("Invalid input. Please enter Y or N.")
    # The loop will continue prompting the user for input
  }
}

    
RemainingCutSize  
Output_gcode_df


    
    

The stock diameter has been reduced to:  0.2825 : The roughing cut size is: 0.015 . The final diameter will be: 0.2545 
 Enter Y to make one last rough cut. Or enter N to go directly to final cut options. y


[1] "You entered Yes (Y)"
[1] "The new diameter is  0.2675" "The new diameter is  -1"    


[1] 0.2675

X,Z
<dbl>,<dbl>
0.4925,-1
0.4775,-1
0.4625,-1
0.4475,-1
0.4325,-1
0.4175,-1
0.4025,-1
0.3875,-1
0.3725,-1


# Input:

In [16]:
Output_gcode_df

X,Z
<dbl>,<dbl>
0.4925,-1
0.4775,-1
0.4625,-1
0.4475,-1
0.4325,-1
0.4175,-1
0.4025,-1
0.3875,-1
0.3725,-1


## Part - The finish cut(s)


In [17]:
valid_input <- FALSE

while (!valid_input) {
  # Prompt the user for input
    
    RemainingCutSize <- format(RemainingCutSize, nsmall = 3)
    
    user_input <<- readline(paste("The current diameter is:" , RemainingCutSize, ". The final diameter is :", format(EndDia, nsmall = 3), ":  Enter Y or N to make a final cut:" ))
    
      # Convert input to uppercase for case-insensitive comparison
      user_input <- toupper(user_input)
  
      # Check if the input is 'Y' or 'N'
  
    if (user_input == 'Y') {      
      
        user_input <<- readline(paste("Enter the size of this finish cut (.0xx) :" ))
        FinisCut <- as.numeric(user_input)

        # print("Part should have a dimension of:")

        RemainingCutSize <- as.numeric(RemainingCutSize)
        RemainingCutSize <- RemainingCutSize - FinisCut     # Taking finish cut  
        format(RemainingCutSize, nsmall = 3) 
      
        # Create a new row for each iteration with X and Z values and add it to the dataframe
         new_row <- data.frame(X = RemainingCutSize, Z = Leng)           
         paste("The finished stock diameter is to be: ", RemainingCutSize)
         Output_gcode_df <- rbind(result_df, new_row) 
   
      
        
        
    } else if(user_input == "N") {
        print("This notebook has completed its operation.  Write the file and then move on to gcode_Part3.ipynb")
        break
       } # end of first Y
   }

RemainingCutSize  # print current diameter 

  
#  Restart kernel and run again with any data entry errors  

The current diameter is: 0.2675 . The final diameter is : 0.2545 :  Enter Y or N to make a final cut: y
Enter the size of this finish cut (.0xx) : .013
The current diameter is: 0.2545 . The final diameter is : 0.2545 :  Enter Y or N to make a final cut: n


[1] "This notebook has completed its operation.  Write the file and then move on to gcode_Part3.ipynb"


[1] "0.2545"

# Input:

In [18]:
# # Ask the user to enter Y or N
# user_input <- readline(prompt = "Enter Y or N: ")

# # Convert user input to uppercase for case insensitivity
# user_input <- toupper(user_input)

# # Check if the user input is Y or N and set the Boolean variable accordingly
# if (user_input == "Y") {
#   answer <- TRUE
# } else if (user_input == "N") {
#   answer <- FALSE
# } else {
#   # Handle cases other than Y or N
#   stop("Invalid input. Please enter Y or N.")
# }

# # Check the value of the Boolean variable using an if-else statement
# if (answer == TRUE) {
#   print("The answer is Yes")
# } else {
#   print("The answer is No")
# }


### Restart kernel and run again with any data entry errors

## Okay ... over the hump

### Note: the number of repetitions will be important in later parts of this process

#### Save this number to a .csv file to be used later


In [19]:
repetitions <- nrow(Output_gcode_df)
repetitions
writeLines(as.character(repetitions), "repetitions.txt")

[1] 16

## Write the df to .csv file

In [20]:
# Write the updated gcode data to a new .csv file

write.csv(Output_gcode_df, file = "gcode_file.csv", row.names = FALSE)

In [21]:
# Read the data from the existing .csv file
gcode <- read.csv("gcode_file.csv", header = T)
gcode

X,Z
<dbl>,<int>
0.4925,-1
0.4775,-1
0.4625,-1
0.4475,-1
0.4325,-1
0.4175,-1
0.4025,-1
0.3875,-1
0.3725,-1


## Done !!!

#### Move on to next file: gcode_Part2.ipynb

In [22]:
stopper

ERROR: Error in eval(expr, envir, enclos): object 'stopper' not found



# Example variable
my_variable <- 10

# Display the value of my_variable in the prompt message
user_input <- readline(paste("Enter a value (current value of my_variable is", my_variable, "): "))

# Show the user input
print(user_input)

RemainingCutSize <- result_df$X[n]
RemainingCutSize



LastCuts <<- readline(paste("The current diameter is:" , RemainingCutSize, ": Cut size is:", Cut, "Make one more ", Cut, "sized cut? Enter Y for yes" ))
    





    
    LastCuts <- as.numeric(LastCuts)
    str(LastCuts)
    RemainingCutSize <- RemainingCutSize - Cut
    RemainingCutSize



FinalCut  <- readline(paste("The diameter has been reduced to:", testCutSize, ": Enter value (.0xx) for final cut: "))
     FinalCut <- as.numeric(FinalCut)
FinalCut <- testCutSize - FinalCut
testCutSize




FinalCut 
as.character(FinalCut) 

FinalCut <- as.numeric(unlist(strsplit(FinalCut, ", ")))   # Convert Char to numeric data type
is.numeric(FinalCut)





In [ ]:
testCutSize <- result_df$X[n]


# LastCut <<- readline(paste("The finish cut(s) loop. Diameter is:" , testCutSize, ": Enter next cut size or 0 to quit: "))
    
if (!is.na(FinalCut)) {
    if (FinalCut >= 0 ) {
        print(paste("final cut is: ", FinalCut))
    } else {
        print("FinalCut is something eles")
    }
} else {
    print("wrong input")
    }
   

## Try converting the df to string, 

In [ ]:
#  Adding a column to an existing dataframe

df_X <- cbind(gCode = "G1", Output_gcode_df)
df_X


In [ ]:
vector_X <- df_X$X
vector_Z <- df_X$Z

In [ ]:
vector_X

In [ ]:
# d <- data.frame(x = 1, y = 1:10, char = char))
testDF <- data.frame(gcode = "G01", X = "X", Xg = vector_X, Z = "Z", Zg = vector_Z, F = "F", Speed = Feed)

In [ ]:
string_list <- list()

for(i in 1:nrow(testDF)) {
    row_string <- paste(testDF[i, ], collapse = " ")  # Concatenate row elements into a string
    string_list <- c(string_list, list(row_string))   # Add the row string to the list
    }


In [ ]:
# Need to seperate the list by rows
# Combine the single string with newline characters


combined_string <- paste(string_list, collapse = "\n")
combined_string
is.list(combined_string)

str(combined_string)

In [ ]:
# updated_list <- c(string_list, new_string)

full_string <- c(combined_string, G0_gcode1)

In [ ]:
# Define the file path and name for the output .csv file
file_path <- "gcode_file.csv"

# Write the dataframe to a .csv file
# write.csv(text = combined_string, file = file_path, row.names = FALSE)    # this worked, then it didn't

# Write the string to a .csv file using writeLines()
writeLines(text = full_string, con = file(file_path))

In [ ]:
file_path <- "gcode_file.csv"

# Open the file in append mode using a file connection
file_conn <- file(file_path, open = "a")

# Append the text to the file using writeLines()
writeLines(text = full_string, file_conn)

# Close the file connection
close(file_conn)

In [ ]:
print(paste("X", format(RadDROzero, nsmall = 3), "Z", format(Leng, nsmall = 2)))
    

###  What do I want?  Add G01 to each line.  Add X and Z.  

### This will involve merging or binding columns

In [ ]:
df_X

### Extract the columns

    It's far easier to create a new df from scratch than to modify an existing df

## Build a new G01 data.frame using the X Z vectors

    It far easier to build a df from scratch than to modify an existing df

In [ ]:
testDF    # the new df with all the components for the gcode G01 
nrow(testDF)

##  Unpack the df again

## Looks Right - note this is a list

In [ ]:
string_list
length(string_list)
is.list(string_list)


### This is NOT a list.  Is just a character vector, aka string

## Append the G00 character vector to the G01 character vector

In [ ]:
# Create a new row - the G00 line, to be inserted into the existing string

new_row <- c("G00", "X", ".250", "Z",".75", "F", Feed*2)  # Replace "Data" with your content
new_row
G0_gcode <- paste(new_row, collapse = " ")
G0_gcode
G0_gcode1 <- paste(G0_gcode, collapse = "\n")     
G0_gcode1
str(G0_gcode1)

In [ ]:
full_string
is.list(full_string)
str(full_string)

### The addtion of G00 was easy when using vectors, aka strings, rather than trying to mod a df

## All looks good

In [ ]:
xexit 

In [ ]:
?writeLines

### Other code that didn't work out

In [ ]:

# New row of text to insert
new_row <- c("G00", "X", ".250", "Z",".75", "F", Feed*2)  # Replace "Data" with your content
new_row
G0_gcode <- paste(new_row, collapse = " ")
G0_gcode
G0_gcode1 <- paste(G0_gcode, collapse = "\n")
G0_gcode1

In [ ]:
# Write the updated data to a new .csv file
write.csv(G0_gcode1, file = "gcode_file.csv", append = TRUE, row.names = FALSE)  # Replace "updated_data.csv" with your desired file name

In [ ]:
# Determine the position to insert the new row (for example, after the 3rd row)
insert_index <- 2

# Insert the new row into the existing data
updated_data <- rbind(existing_data[1:insert_index, ], 
                new_row, existing_data[(insert_index + 1):nrow(existing_data), ])

In [ ]:
# Your character string
my_string <- "'G01'' ''X''0.235''Z''-0.75'"

# Remove ' character using gsub
cleaned_string <- gsub("'", "", my_string)

# Print the cleaned string
print(cleaned_string)

In [ ]:
testDF[1,]
gcode <- as.character(testDF[1,])
gcode
is.character(gcode)
is.vector(gcode)


In [ ]:
str(cleaned_string)